In [1]:
from learner import Learner

import os

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import librosa
import numpy as np
import torch
from PIL import Image
from sklearn.utils import shuffle
from tqdm import tqdm
from copy import deepcopy
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

2023-08-23 17:18:45,998 - butterfly_op.py[line:23] - WARNING: Import universal_cuda fail
2023-08-23 17:18:46,000 - butterfly_op.py[line:27] - WARNING: Import hadamard_cuda fail
2023-08-23 17:18:46,002 - matrix_parametrization.py[line:23] - WARNING: Cannot import matrix_parametrization_cuda. Decomposers can only work on CPU mode


In [2]:
config = [
    ('conv2d', [32, 1, 7, 7, 1, 3]),
    ('relu', [True]),
    ('bn', [32]),
    ('conv2d', [16, 32, 5, 5, 1, 2]),
    ('relu', [True]),
    ('bn', [16]),
    ('conv2d', [8, 16, 3, 3, 1, 1]),
    ('relu', [True]),
    ('bn', [8]),
    ('flatten', []),
    ('linear', [5, 8 * 28 * 28])
]

In [13]:
from birdCallNShot import BirdCallNShot

db_train = BirdCallNShot('birdCall',
                       batchsz=32,
                       n_way=5,
                       k_shot=10,
                       k_query=15,
                       SPEC_SHAPE = (28,28))

DB: train (99, 100, 28, 28, 1) test (33, 100, 28, 28, 1)


In [14]:
x_spt, y_spt, x_qry, y_qry = map(lambda x: torch.from_numpy(x).to(torch.device('cpu')), db_train.next())
device = torch.device('cpu')

In [15]:
def check_accuracy(data, labels, model, idx = 0):
    # print('Checking accuracy on set')   
    num_correct = 0
    num_samples = 0
    num_data_points = data.shape[0]
    
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        x, y = data[idx], labels[idx]
        x = x.to(device=torch.device("cpu"), dtype=torch.float32)  # move to device, e.g. GPU
        y = y.to(device=torch.device("cpu"), dtype=torch.long)
        scores = model(x)
        _, preds = scores.max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        # print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc
        
def train(data, labels, model, optimizer, epochs=50, idx = 0):
    model = model.to(device=torch.device("cpu"))  # move the model parameters to CPU/GPU    
    test_accuracies = [check_accuracy(x_qry, y_qry, model, idx)]
    for e in range(epochs):
        # print(f"Epoch number: {e}")
        x, y = data[idx], labels[idx]
        model.train()  # put model to training mode
        x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
        y = y.to(device=device, dtype=torch.long)

        scores = model(x)
        loss = F.cross_entropy(scores, y)

        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad()

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()

        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        
        
        optimizer.step()

        param_idx = 0
        
        test_accuracies.append(check_accuracy(x_qry, y_qry, model, idx))
    return test_accuracies

In [17]:
import datetime

lr_list = [0.01]

print("doing one shot maml accuracies now")
one_shot_test_accuracies = {}

for lr in lr_list:
    models = []
    models = [Learner(config, 1, 28, mb = 16) for _ in range(16)]
    for i in range(16):
        models[i].load_state_dict(torch.load('onn_birdcall.pth'))
    test_accs = []
    for idx in range(16): 
        if idx % 8 == 0:
            print(f"Time is {datetime.datetime.now()}, idx = {idx}")
        models[idx].eval()
        optimizer = optim.SGD(models[idx].parameters(), lr=lr, momentum=0.9, nesterov=True)
        test_accs.append(train(x_spt, y_spt, models[idx], optimizer, epochs = 20, idx = idx))
    one_shot_test_accuracies[str(lr)] = np.average(np.array(test_accs), axis = 0)
    print(f"For LR = {lr}: {one_shot_test_accuracies[str(lr)]}")

print("doing scratch accuracies now")
scratch_test_accuracies = {}

for lr in lr_list:
    models = []
    models = [Learner(config, 1, 28, mb = 16) for _ in range(16)]
    test_accs = []
    for idx in range(16):
        if idx % 8 == 0:
            print(f"Time is {datetime.datetime.now()}, idx = {idx}")
        models[idx].eval()
        optimizer = optim.SGD(models[idx].parameters(), lr=lr, momentum=0.9, nesterov=True)
        test_accs.append(train(x_spt, y_spt, models[idx], optimizer, epochs = 20, idx = idx))
    scratch_test_accuracies[str(lr)] = np.average(np.array(test_accs), axis = 0)
    print(f"For LR = {lr}: {scratch_test_accuracies[str(lr)]}")

doing one shot maml accuracies now
Time is 2023-08-24 14:43:21.111624, idx = 0
Time is 2023-08-24 14:43:49.576817, idx = 8
For LR = 0.01: [0.2125     0.61333333 0.66583333 0.69416667 0.69333333 0.68083333
 0.67583333 0.66833333 0.66833333 0.65833333 0.66666667 0.6625
 0.6675     0.65916667 0.65916667 0.66166667 0.665      0.6675
 0.66583333 0.6675     0.66666667]
doing scratch accuracies now
Time is 2023-08-24 14:44:21.197443, idx = 0
Time is 2023-08-24 14:44:48.854707, idx = 8
For LR = 0.01: [0.2025     0.22916667 0.26583333 0.29916667 0.31833333 0.3525
 0.36833333 0.37416667 0.3825     0.39833333 0.40666667 0.41333333
 0.41333333 0.41666667 0.42       0.42166667 0.42083333 0.425
 0.42666667 0.42666667 0.42916667]


In [18]:
import matplotlib.pyplot as plt

y0 = scratch_test_accuracies[str(0.003)]
y1 = one_shot_test_accuracies[str(0.003)]
plt.plot(np.array(list(range(0, len(y0)))), y0, label='Trained from Scratch', color='blue')
plt.plot(np.array(list(range(0,len(y1)))), y1, label='Pretrained with Metalearning', color='orange')

# Customize the plot
plt.xlabel('Number of Iterations of Training')
plt.ylabel('Test Classification Accuracy')
plt.title('Ten Shot Bird Call Classification: \nComparing Metalearned ONN vs. Randomly Initialized ONN')
plt.grid(True)
plt.legend()
plt.savefig('onn_ten_shot_birdcall_metalearning_comparison_lr=0.003.jpg', format='jpg')
plt.show()

KeyError: '0.003'